<a href="https://colab.research.google.com/github/TienNguyenMSBA/hotelbooking/blob/main/Copy_of_Group_Project_1__Advance_Machine_Learning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### README

This notebook contains the code to predict final demand of a hotel booking data using different models:

1. Advance Booking (Pick-up) Models (Additive and Multiplicative)
2. Advance Booking Feedforward Neural Network Models
3. Time Series LSTM Models


In [ ]:
import numpy as np
import pandas as pd
from datetime import timedelta
from tqdm import tqdm
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving hotel_bookings.csv to hotel_bookings.csv


##### Read the data (csv file)

In [ ]:
df = pd.read_csv("hotel_bookings.csv")
df

,hotel_code,stay_date,booking_date,days_prior,bookings_on_hand,final_demand
0,G,5/1/2008,5/2/2007,365,0,116
1,G,5/1/2008,5/3/2007,364,0,116
2,G,5/1/2008,5/4/2007,363,0,116
3,G,5/1/2008,5/5/2007,362,0,116
4,G,5/1/2008,5/6/2007,361,0,116
...,...,...,...,...,...,...
801535,W,4/30/2010,4/26/2010,4,21,28
801536,W,4/30/2010,4/27/2010,3,21,28
801537,W,4/30/2010,4/28/2010,2,22,28
801538,W,4/30/2010,4/29/2010,1,24,28


##### **1. Pick-up models**

##### Convert date columns to datetime for filtering

In [ ]:
## Define features (X) and target (y)
feature_cols = ["days_prior", "bookings_on_hand", "day_of_week", "month", "naive_forecast"]
target_col = "final_demand"

def convert_datetime(df, columns):
    for col in columns:
        df[col] = pd.to_datetime(df[col])
    return df

date_columns = ["stay_date", "booking_date"]
df = convert_datetime(df, date_columns)

## Create feature columns used by models and aggregations
df["day_of_week"] = df["stay_date"].dt.day_name()
df["month"] = df["stay_date"].dt.month
df['days_prior_range'] = pd.cut(df['days_prior'], bins=[0,7,14,21,30], labels=['1-7','8-14','15-21','22-30'], right=True)
df['remaining_demand'] = df['final_demand'] - df['bookings_on_hand']

## Create naive forecast based on previous year's final_demand for same hotel and stay_date
df['stay_date'] = pd.to_datetime(df['stay_date'])
df = df.sort_values(['hotel_code', 'stay_date'])
# Create a shifted date column for lookup
df['prev_year_date'] = df['stay_date'] - pd.DateOffset(years=1)
# Build a mapping from (hotel_code, date) → final_demand
key_to_demand = dict(zip(zip(df['hotel_code'], df['stay_date']), df['final_demand']))
# Lookup previous year's value efficiently
df['naive_forecast'] = [
    key_to_demand.get((h, prev_date)) for h, prev_date in zip(df['hotel_code'], df['prev_year_date'])
]
# Drop missing
df = df.dropna(subset=['naive_forecast'])



In [ ]:
df

,hotel_code,stay_date,booking_date,days_prior,bookings_on_hand,final_demand,day_of_week,month,days_prior_range,remaining_demand,prev_year_date,naive_forecast
133590,G,2009-05-01,2008-05-01,365,0,99,Friday,5,NaN,99,2008-05-01,116.0
133591,G,2009-05-01,2008-05-02,364,0,99,Friday,5,NaN,99,2008-05-01,116.0
133592,G,2009-05-01,2008-05-03,363,0,99,Friday,5,NaN,99,2008-05-01,116.0
133593,G,2009-05-01,2008-05-04,362,0,99,Friday,5,NaN,99,2008-05-01,116.0
133594,G,2009-05-01,2008-05-05,361,0,99,Friday,5,NaN,99,2008-05-01,116.0
...,...,...,...,...,...,...,...,...,...,...,...,...
801535,W,2010-04-30,2010-04-26,4,21,28,Friday,4,1-7,7,2009-04-30,59.0
801536,W,2010-04-30,2010-04-27,3,21,28,Friday,4,1-7,7,2009-04-30,59.0
801537,W,2010-04-30,2010-04-28,2,22,28,Friday,4,1-7,6,2009-04-30,59.0
801538,W,2010-04-30,2010-04-29,1,24,28,Friday,4,1-7,4,2009-04-30,59.0


In [ ]:
## Split the data into training and validation sets based on stay_date
train = df[df["stay_date"] < "2010-02-01"]
val = df[df["stay_date"] >= "2010-02-01"]

##### **Helper functions** for pick-up models
- Additive model (bookings_on_hand + average remaining demand)
- Multiplicative model (bookings_on_hand/average booking rate)
- Calculate error metrics (MAE, MAPE)
- Compute MASE using naive forecast (previous year). For example, the naive forecast for stay date Apr 1st, 2010 is the actual demand of Apr 2nd, 2009
- Add the forecast to the validation set, using days_prior 1-30
- Print out the forecasting errors grouped by hotel code, days_prior range, and month

In [ ]:
## Additive model (bookings_on_hand + average remaining demand)
def add_model(df):
    df = df.copy()
    df = df[df['days_prior'].between(1,30)]
    df['average_remaining_demand'] = df.groupby(['hotel_code', 'days_prior'], sort=False)[
        'remaining_demand'].transform(np.mean)
    df['forecast'] = df['bookings_on_hand'] + df['average_remaining_demand']
    return df

## Multiplicative model (bookings_on_hand/average booking rate)
def multi_model(df):
    df = df.copy()
    df = df[df['days_prior'].between(1,30)]
    df['booking_rate'] = df['bookings_on_hand'] / df['final_demand']
    df['average_booking_rate'] = df.groupby(['hotel_code', 'days_prior'], sort=False)['booking_rate'].transform(np.mean)
    df['forecast'] = df['bookings_on_hand'] / df['average_booking_rate']

    # Handle inf/nan
    df['forecast'] = df['forecast'].replace([np.inf, -np.inf], np.nan)
    df['forecast'] = df['forecast'].fillna(df['bookings_on_hand'])
    return df

## Calculate error metrics (MAE, MAPE)
def compute_errors(df, naive_df=None):
    df = df.copy()
    df['error'] = df['final_demand'] - df['forecast']
    df['abs_error'] = df['error'].abs()
    df['mae'] = df['abs_error']
    df['mape'] = df['abs_error'] / df['final_demand'].replace(0, np.nan) * 100

        # --- Compute MASE ---
    if 'naive_forecast' in df.columns:
        # MAE of model
        mae_model = df['abs_error'].mean()

        # MAE of naive forecast
        df['naive_error'] = (df['final_demand'] - df['naive_forecast']).abs()
        mae_naive = df['naive_error'].mean()

        # Scaled error
        df['mase'] = df['abs_error'] / mae_naive if mae_naive != 0 else np.nan
    else:
        df['mase'] = np.nan
    return df


## Function for adding the forecast to the validation set, using days_prior 1-30
def validation(train, val, pickup, model_type='add'):
        # Merge pickup info by days_prior + hotel_code
    val = val[val['days_prior'].between(1, 30)].copy()
    booking_info = train[['days_prior', 'hotel_code', pickup]].drop_duplicates()
    forecast = val.merge(booking_info, on=['hotel_code', 'days_prior'], how='left')

    # Compute forecast based on model type
    if model_type == 'add':
        forecast['forecast'] = forecast['bookings_on_hand'] + forecast[pickup].fillna(0)
    elif model_type == 'multi':
        forecast['forecast'] = (forecast['bookings_on_hand'] / forecast[pickup].replace(0, np.nan))
    else:
        raise ValueError("Invalid model_type. Use 'add' or 'multi'.")

    # Compute errors relative to naive model (previous year's actuals)
    forecast = compute_errors(forecast, naive_df=train)

    return forecast

## Print out the forecasting errors grouped by hotel code, days_prior range, and month
def summarize_errors(df):
    # Overall metrics
    print(f"\nOVERALL METRICS:")
    print(f"MAE:  {df['mae'].mean():.2f}")
    print(f"MAPE: {df['mape'].mean():.2f}%")
    print(f"MASE: {df['mase'].mean():.2f}")

    # Group by month and hotel
    monthly_errors = (
        df.groupby(['hotel_code', 'month'])
        [['mae', 'mape', 'mase']]
        .mean()
        .reset_index()
    )

    # Group by days-prior range and hotel
    daysprior_errors = (
        df.groupby(['hotel_code', 'days_prior_range'])
        [['mae', 'mape', 'mase']]
        .mean()
        .reset_index()
    )

    # Display results
    for hotel in df['hotel_code'].unique():
        print(f"\n===== {hotel} =====")
        print("\nAverage Errors by Month:")
        print(
            monthly_errors[monthly_errors['hotel_code'] == hotel]
            .drop(columns='hotel_code')
            .rename(columns={'month': 'Month', 'mae': 'Average MAE', 'mape': 'Average MAPE', 'mase': 'Average MASE'})
            .to_string(index=False)
        )

        print("\nAverage Errors by Days-Prior Range:")
        print(
            daysprior_errors[daysprior_errors['hotel_code'] == hotel]
            .drop(columns='hotel_code')
            .rename(columns={'days_prior_range': 'Days-Prior Range', 'mae': 'Average MAE', 'mape': 'Average MAPE', 'mase': 'Average MASE'})
            .to_string(index=False)
        )

##### Apply the functions above to the dataframe in training and validation data

In [ ]:
## Train the data and compute errors in train
train_add = add_model(train)
train_add = compute_errors(train_add)

train_multi = multi_model(train)
train_multi = compute_errors(train_multi)

## Validate models with test/validation data
val_add = validation(train_add, val, 'average_remaining_demand', model_type='add')
val_multi = validation(train_multi, val, 'average_booking_rate', model_type='multi')


/tmp/ipython-input-2059195548.py:6: FutureWarning: The provided callable <function mean at 0x796b1116c220> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  'remaining_demand'].transform(np.mean)
/tmp/ipython-input-2059195548.py:15: FutureWarning: The provided callable <function mean at 0x796b1116c220> is currently using SeriesGroupBy.mean. In a future version of pandas, the provided callable will be used directly. To keep current behavior pass the string "mean" instead.
  df['average_booking_rate'] = df.groupby(['hotel_code', 'days_prior'], sort=False)['booking_rate'].transform(np.mean)


#####

##### Results from ADDITIVE model with forcasting errors below

In [ ]:
summarize_errors(val_add)


OVERALL METRICS:
MAE:  22.24
MAPE: 37.27%
MASE: 0.65

===== G =====

Average Errors by Month:
 Month  Average MAE  Average MAPE  Average MASE
     2    10.925664     12.325796      0.318466
     3    14.028950     15.561829      0.408922
     4    15.672351     16.125399      0.456825

Average Errors by Days-Prior Range:
Days-Prior Range  Average MAE  Average MAPE  Average MASE
             1-7    10.299253     11.068720      0.300207
            8-14    13.981000     14.945554      0.407525
           15-21    14.494626     15.583656      0.422496
           22-30    15.197063     16.758450      0.442971

===== M =====

Average Errors by Month:
 Month  Average MAE  Average MAPE  Average MASE
     2    29.821058     43.083934      0.869238
     3    33.054909     41.830497      0.963500
     4    34.602810     48.776133      1.008619

Average Errors by Days-Prior Range:
Days-Prior Range  Average MAE  Average MAPE  Average MASE
             1-7    18.101920     24.192507      0.527643


/tmp/ipython-input-2059195548.py:85: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['hotel_code', 'days_prior_range'])


##### Results from MULTIPLICATIVE model with forcasting errors below

In [ ]:
summarize_errors(val_multi)


OVERALL METRICS:
MAE:  32.13
MAPE: 38.22%
MASE: 0.94

===== G =====

Average Errors by Month:
 Month  Average MAE  Average MAPE  Average MASE
     2    32.845784     38.082843      0.957404
     3    26.939458     29.703438      0.785244
     4    36.902783     39.491328      1.075660

Average Errors by Days-Prior Range:
Days-Prior Range  Average MAE  Average MAPE  Average MASE
             1-7    16.180277     17.808441      0.471630
            8-14    30.400597     33.695502      0.886131
           15-21    36.938319     40.945533      1.076696
           22-30    42.227468     46.891332      1.230866

===== M =====

Average Errors by Month:
 Month  Average MAE  Average MAPE  Average MASE
     2    31.620944     33.680872      0.921702
     3    51.958347     48.311743      1.514506
     4    31.758991     34.957476      0.925726

Average Errors by Days-Prior Range:
Days-Prior Range  Average MAE  Average MAPE  Average MASE
             1-7    15.466295     15.410372      0.450819


/tmp/ipython-input-2059195548.py:85: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['hotel_code', 'days_prior_range'])


##### **2. Define Neural Network Model**

In [ ]:
###########################################################################
### Define Neural Network Model
###########################################################################

class Model(nn.Module):  ## subclass nn.Module
    ## input size: number of features (X variables), dimension of X
    ## hidden-size: number of nodes in hidden layers
    ## output_size: dimension of y
    def __init__(self, input_size, hidden_size, output_size):
        super().__init__()
        ## nn.Linear: a linear transformation,a.k.a a fully connected layer or a dense layer to implement summation junction
        ## define two hidden layers, layer1 and layer2
        self.layer1 = nn.Linear(input_size, hidden_size)   ## layer 1 has "input_size" number of input nodes and "hidden_size" number of output nodes
        self.layer2 = nn.Linear(hidden_size, output_size)  ## layer 2 has "hidden_size" number of input nodes and "output_size" number of output nodes
        self.activation = nn.ReLU()  # Use ReLU for hidden layer

    ## forward() method that must be defined within any class that subclasses torch.nn.Module.
    ## This method defines the computational graph of a neural network,
    ## outlining how input data is processed through the network's layers to produce an output.
    def forward(self, x):
        x = self.activation(self.layer1(x))  # Apply ReLU to hidden layer
        x = self.layer2(x)                   # Linear output for regression
        return x

In [ ]:
###########################################################################
### Helper functions, validation with days prior 1-30
###########################################################################

def prepare_data(train_df, val_df, feature_cols, target_col):
    """Prepare train and val sets"""
    val_df = val_df[val_df['days_prior'].between(1, 30)].copy()
    X_train = pd.get_dummies(train_df[feature_cols], columns=["day_of_week"], drop_first=True)
    X_val = pd.get_dummies(val_df[feature_cols], columns=["day_of_week"], drop_first=True)
    X_train, X_val = X_train.align(X_val, join="left", axis=1, fill_value=0)

    y_train = train_df[target_col].to_numpy().astype(np.float32)
    y_val = val_df[target_col].to_numpy().astype(np.float32)

    return X_train.to_numpy().astype(np.float32), y_train, X_val.to_numpy().astype(np.float32), y_val

##### Normalize data, convert to tensors, create DataLoader, setup loss function and optimizer, train model, and evaluate

In [ ]:
def train_model(X_train, y_train, X_test, y_test, num_epochs=10, hidden_size=20, lr=0.001):
    """Train and evaluate model for one hotel."""
    # Normalize
    X_train_norm = (X_train - np.mean(X_train)) / np.std(X_train)
    X_test_norm = (X_test - np.mean(X_train)) / np.std(X_train)

    # Convert to tensors
    X_train_t = torch.from_numpy(X_train_norm).float()
    y_train_t = torch.from_numpy(y_train).float()
    X_test_t = torch.from_numpy(X_test_norm).float()
    y_test_t = torch.from_numpy(y_test).float()

    # Create DataLoader
    train_ds = TensorDataset(X_train_t, y_train_t)
    train_dl = DataLoader(train_ds, batch_size=20, shuffle=True)

    # Model setup
    input_size = X_train_t.shape[1]
    model = Model(input_size, hidden_size, 1)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    # Train
    model.train()
    for epoch in range(num_epochs):
        epoch_loss = 0
        for xb, yb in train_dl:
            pred = model(xb)
            loss = loss_fn(pred.squeeze(), yb)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            epoch_loss += loss.item()
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {epoch_loss/len(train_dl):.4f}")

    # Evaluate
    model.eval()
    with torch.no_grad():
        preds = model(X_test_t).squeeze()
        mse = loss_fn(preds, y_test_t)
        rmse = torch.sqrt(mse).item()

    print(f"Validation RMSE: {rmse:.4f}")
    return model, preds.numpy(), rmse


##### Loop through hotels and run model + error summaries

In [ ]:
###########################################################################
### Loop through hotels and run model + error summaries
###########################################################################
all_results = []

for hotel in df['hotel_code'].unique():
    print(f"\n== {hotel} ==")
    train_hotel = train[train['hotel_code'] == hotel]
    val_hotel = val[val['hotel_code'] == hotel]

    if len(train_hotel) < 10 or len(val_hotel) < 10:
        print("Not enough data, skipping...")
        continue

    # Prepare data
    X_train, y_train, X_val, y_val = prepare_data(train_hotel, val_hotel, feature_cols, target_col)

    # Train model
    model, y_pred, rmse = train_model(X_train, y_train, X_val, y_val)


    # Compute and store errors
    val_hotel = val_hotel[val_hotel['days_prior'].between(1, 30)].copy()
    val_hotel = val_hotel.copy()
    val_hotel['forecast'] = y_pred
    val_hotel['pred'] = y_pred
    val_hotel['abs_error'] = np.abs(val_hotel['pred'] - val_hotel['final_demand'])
    val_hotel['mae'] = val_hotel['abs_error']
    val_hotel['mape'] = val_hotel['abs_error'] / (val_hotel['final_demand'] + 1e-8)
    val_hotel['rmse'] = rmse

    # Compute MASE
    val_hotel = compute_errors(val_hotel)

    # Append
    all_results.append(val_hotel)

# Combine all hotel results
final_results = pd.concat(all_results, ignore_index=True)
print("\n All models: ")

# Summarize errors across all hotels
summarize_errors(final_results)


== G ==
Epoch 1/10, Loss: 1516.9418
Epoch 2/10, Loss: 579.7858
Epoch 3/10, Loss: 568.1127
Epoch 4/10, Loss: 559.9127
Epoch 5/10, Loss: 553.4039
Epoch 6/10, Loss: 547.9973
Epoch 7/10, Loss: 542.4481
Epoch 8/10, Loss: 537.2058
Epoch 9/10, Loss: 532.0549
Epoch 10/10, Loss: 526.6719
Validation RMSE: 19.1622

== M ==
Epoch 1/10, Loss: 2952.3128
Epoch 2/10, Loss: 2081.4788
Epoch 3/10, Loss: 2061.8878
Epoch 4/10, Loss: 2057.0109
Epoch 5/10, Loss: 2054.9488
Epoch 6/10, Loss: 2053.5868
Epoch 7/10, Loss: 2052.0244
Epoch 8/10, Loss: 2050.6340
Epoch 9/10, Loss: 2049.4054
Epoch 10/10, Loss: 2047.4157
Validation RMSE: 41.6279

== W ==
Epoch 1/10, Loss: 1599.0564
Epoch 2/10, Loss: 1179.9912
Epoch 3/10, Loss: 1163.9650
Epoch 4/10, Loss: 1156.6413
Epoch 5/10, Loss: 1152.0172
Epoch 6/10, Loss: 1148.5114
Epoch 7/10, Loss: 1145.2872
Epoch 8/10, Loss: 1141.5951
Epoch 9/10, Loss: 1137.5933
Epoch 10/10, Loss: 1132.9393
Validation RMSE: 27.6597

 All models: 

OVERALL METRICS:
MAE:  24.86
MAPE: 45.64%
MASE: 

/tmp/ipython-input-2059195548.py:85: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['hotel_code', 'days_prior_range'])



##### **3. Time Series LSTM Model**

In [ ]:
###########################################################################
### Define neural network model class by subclassing nn.Module
###########################################################################
##### Define a LSTM Model - LSTM layer + linear layer

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout=0.2):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.linear = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.linear(out[:, -1, :])
        return out

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') ## get the device information
print(device)

cuda


##### Reshape data and convert to tensor

In [ ]:
def create_sequences(data, lookback=100):
    X, y = [], []
    for i in range(len(data) - lookback):
        X.append(data[i:i+lookback])
        y.append(data[i+lookback])
    return np.array(X), np.array(y)

def prepare_data(df, hotel_code):
    hotel_df = df[df["hotel_code"] == hotel_code].copy()
    hotel_df = hotel_df.set_index("stay_date").asfreq("D").fillna(method="ffill")
    scaler = MinMaxScaler()
    scaled = scaler.fit_transform(hotel_df[["final_demand"]])
    return hotel_df, scaled, scaler

##### Loop through each hotel and apply Time Series LSTM

In [ ]:
# Filter relevant columns
df = df[["stay_date", "final_demand", "hotel_code", "naive_forecast"]].drop_duplicates()


# Split train/test
train_df = df[df["stay_date"] < "2010-01-01"]
val_df  = df[(df["stay_date"] >= "2010-01-01") & (df["stay_date"] <= "2010-04-30")]


# Combine for continuous indexing (needed for lookback)
df = pd.concat([train_df, val_df]).sort_values("stay_date")


In [ ]:
input_size = 1
output_size = 1
dropout = 0
lookback = 60
batch_size = 64
num_epochs = 60
learning_rate = 0.001
forecast_horizon = 30
hidden_size = 10


# ==== Hotel-Specific LSTM Configurations ====
hotel_configs = {
    "G": {"hidden_size": 120, "num_layers": 3},
    "M": {"hidden_size": 30, "num_layers": 1},
    "W": {"hidden_size": 90, "num_layers": 2},
}

In [ ]:
for hotel, cfg in hotel_configs.items():
    hidden_size = cfg["hidden_size"]
    num_layers = cfg["num_layers"]

    print(f"Initializing LSTM for hotel {hotel} (hidden={hidden_size}, layers={num_layers})")
    model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout)
    print(model)

Initializing LSTM for hotel G (hidden=120, layers=3)
LSTMModel(
  (lstm): LSTM(1, 120, num_layers=3, batch_first=True)
  (linear): Linear(in_features=120, out_features=1, bias=True)
)
Initializing LSTM for hotel M (hidden=30, layers=1)
LSTMModel(
  (lstm): LSTM(1, 30, batch_first=True)
  (linear): Linear(in_features=30, out_features=1, bias=True)
)
Initializing LSTM for hotel W (hidden=90, layers=2)
LSTMModel(
  (lstm): LSTM(1, 90, num_layers=2, batch_first=True)
  (linear): Linear(in_features=90, out_features=1, bias=True)
)


In [ ]:


# ==== Forecast Date Range ====
asof_dates = pd.date_range("2010-01-30", "2010-03-31", freq="D")
results = []

# ===== Loop through each hotel =====
for hotel in df["hotel_code"].unique():
    print(f"\n🏨 Training once for hotel {hotel} ...")

    # --- Use hotel-specific configuration ---
    config = hotel_configs.get(hotel, {"hidden_size": 64, "num_layers": 2})
    hidden_size = config["hidden_size"]
    num_layers = config["num_layers"]

    # --- Prepare & scale data ---
    hotel_df, scaled, scaler = prepare_data(df, hotel)
    hotel_df["scaled_demand"] = scaled.flatten()

    # --- Split train/validation by time ---
    train_df = hotel_df[hotel_df.index < "2010-01-01"]
    val_df = hotel_df[(hotel_df.index >= "2010-01-01") & (hotel_df.index <= "2010-04-30")]

    train_series = train_df["scaled_demand"].values
    val_series = val_df["scaled_demand"].values

    # --- Create sequences ---
    X_train, y_train = create_sequences(train_series, lookback)
    X_val, y_val = create_sequences(val_series, lookback)

    # --- Convert to tensors ---
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32).unsqueeze(-1)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32).unsqueeze(-1)
    X_val_tensor = torch.tensor(X_val, dtype=torch.float32).unsqueeze(-1)
    y_val_tensor = torch.tensor(y_val, dtype=torch.float32).unsqueeze(-1)

    # --- DataLoader for mini-batch training ---
    train_loader = DataLoader(TensorDataset(X_train_tensor, y_train_tensor), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val_tensor, y_val_tensor), batch_size=batch_size, shuffle=False)

    # --- Model setup ---
    model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout).to(device)
    loss_fn = nn.MSELoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    # --- Training Loop ---
    import time
    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()
        total_train_loss = 0.0
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device)
            optimizer.zero_grad()
            preds = model(batch_X)
            loss = loss_fn(preds, batch_y)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()

        avg_train_loss = total_train_loss / len(train_loader)

        # Validation
        model.eval()
        total_val_loss = 0.0
        with torch.no_grad():
            for batch_Xv, batch_yv in val_loader:
                batch_Xv, batch_yv = batch_Xv.to(device), batch_yv.to(device)
                val_preds = model(batch_Xv)
                val_loss = loss_fn(val_preds, batch_yv)
                total_val_loss += val_loss.item()
        avg_val_loss = total_val_loss / len(val_loader)

        if (epoch + 1) % 10 == 0:
            print(f"Epoch [{epoch + 1}/{num_epochs}] - Train Loss: {avg_train_loss:.5f}, Val Loss: {avg_val_loss:.5f}")

    elapsed_time = time.time() - start_time
    print(f"✅ Done training {hotel}. Time elapsed: {elapsed_time:.2f}s")

    # --- Rolling As-of-Date Forecasts ---
    model.eval()
    demand_series = hotel_df["scaled_demand"].values
    for asof_date in tqdm(asof_dates, desc=f"Rolling forecasts for {hotel}"):
        mask = hotel_df.index <= asof_date
        full_series = demand_series[mask]
        if len(full_series) < lookback:
            continue

        last_input = full_series[-lookback:].reshape(1, lookback, 1)
        last_input = torch.tensor(last_input, dtype=torch.float32).to(device)
        preds = []

        for _ in range(forecast_horizon):
            with torch.no_grad():
                pred = model(last_input).item()
                preds.append(pred)
                new_input = torch.tensor([[[pred]]], dtype=torch.float32).to(device)
                last_input = torch.cat([last_input[:, 1:, :], new_input], dim=1)

        preds_rescaled = scaler.inverse_transform(np.array(preds).reshape(-1, 1)).flatten()
        forecast_dates = pd.date_range(asof_date + pd.Timedelta(days=1),
                                       asof_date + pd.Timedelta(days=forecast_horizon))

        tmp = pd.DataFrame({
            "hotel_code": hotel,
            "as_of_date": asof_date,
            "stay_date": forecast_dates,
            "days_prior": np.arange(1, forecast_horizon + 1),
            "fcst": preds_rescaled
        })
        results.append(tmp)

forecast_df = pd.concat(results).reset_index(drop=True)
print("\n✅ All hotels completed. Forecast dataframe ready.")


🏨 Training once for hotel G ...


/tmp/ipython-input-2720528048.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hotel_df = hotel_df.set_index("stay_date").asfreq("D").fillna(method="ffill")


Epoch [10/60] - Train Loss: 0.05509, Val Loss: 0.06255
Epoch [20/60] - Train Loss: 0.04731, Val Loss: 0.04742
Epoch [30/60] - Train Loss: 0.04414, Val Loss: 0.04469
Epoch [40/60] - Train Loss: 0.04239, Val Loss: 0.03930
Epoch [50/60] - Train Loss: 0.03860, Val Loss: 0.04011
Epoch [60/60] - Train Loss: 0.03874, Val Loss: 0.03893
✅ Done training G. Time elapsed: 1.92s


Rolling forecasts for G: 100%|██████████| 61/61 [00:01<00:00, 33.26it/s]
/tmp/ipython-input-2720528048.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hotel_df = hotel_df.set_index("stay_date").asfreq("D").fillna(method="ffill")



🏨 Training once for hotel W ...
Epoch [10/60] - Train Loss: 0.06940, Val Loss: 0.05598
Epoch [20/60] - Train Loss: 0.06819, Val Loss: 0.05934
Epoch [30/60] - Train Loss: 0.06559, Val Loss: 0.05656
Epoch [40/60] - Train Loss: 0.06543, Val Loss: 0.06275
Epoch [50/60] - Train Loss: 0.06531, Val Loss: 0.05408
Epoch [60/60] - Train Loss: 0.06210, Val Loss: 0.05556
✅ Done training W. Time elapsed: 0.98s


Rolling forecasts for W: 100%|██████████| 61/61 [00:00<00:00, 61.17it/s]
/tmp/ipython-input-2720528048.py:10: FutureWarning: DataFrame.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  hotel_df = hotel_df.set_index("stay_date").asfreq("D").fillna(method="ffill")



🏨 Training once for hotel M ...
Epoch [10/60] - Train Loss: 0.12151, Val Loss: 0.11308
Epoch [20/60] - Train Loss: 0.07054, Val Loss: 0.08169
Epoch [30/60] - Train Loss: 0.06867, Val Loss: 0.07959
Epoch [40/60] - Train Loss: 0.06768, Val Loss: 0.07903
Epoch [50/60] - Train Loss: 0.06745, Val Loss: 0.07869
Epoch [60/60] - Train Loss: 0.06707, Val Loss: 0.07833
✅ Done training M. Time elapsed: 0.62s


Rolling forecasts for M: 100%|██████████| 61/61 [00:00<00:00, 66.06it/s]


✅ All hotels completed. Forecast dataframe ready.


In [ ]:
forecast_df

,hotel_code,as_of_date,stay_date,days_prior,fcst
0,G,2010-01-30,2010-01-31,1,84.835646
1,G,2010-01-30,2010-02-01,2,86.065227
2,G,2010-01-30,2010-02-02,3,87.057619
3,G,2010-01-30,2010-02-03,4,87.813918
4,G,2010-01-30,2010-02-04,5,88.384933
...,...,...,...,...,...
5485,M,2010-03-31,2010-04-26,26,97.070566
5486,M,2010-03-31,2010-04-27,27,97.067845
5487,M,2010-03-31,2010-04-28,28,97.065542
5488,M,2010-03-31,2010-04-29,29,97.063585


In [ ]:
forecast_df = pd.concat(results).reset_index(drop=True)
forecast_df.rename(columns={"fcst": "forecast"}, inplace=True)

# Merge forecasts with actuals by stay_date and hotel_code
forecast_df = forecast_df.merge(
    df[["stay_date", "hotel_code", "final_demand", "naive_forecast"]],
    on=["stay_date", "hotel_code"],
    how="left"
)

forecast_df = compute_errors(forecast_df)
forecast_df["month"] = forecast_df["stay_date"].dt.month
forecast_df['days_prior_range'] = pd.cut(forecast_df['days_prior'], bins=[0,7,14,21,30], labels=['1-7','8-14','15-21','22-30'], right=True)
summarize_errors(forecast_df)



OVERALL METRICS:
MAE:  29.32
MAPE: 50.38%
MASE: 0.86

===== G =====

Average Errors by Month:
 Month  Average MAE  Average MAPE  Average MASE
     1    36.835646     76.740928      1.079872
     2    20.841527     24.524891      0.610989
     3    22.976524     27.734922      0.673579
     4    20.333649     20.219089      0.596100

Average Errors by Days-Prior Range:
Days-Prior Range  Average MAE  Average MAPE  Average MASE
             1-7    21.269696     24.888603      0.623542
            8-14    22.001441     24.579837      0.644993
           15-21    22.131929     25.018530      0.648819
           22-30    21.818329     25.701205      0.639625

===== W =====

Average Errors by Month:
 Month  Average MAE  Average MAPE  Average MASE
     1    36.164209    129.157889      1.060189
     2    25.965371     58.047891      0.761200
     3    21.963667     48.461659      0.643886
     4    27.246030    105.743067      0.798744

Average Errors by Days-Prior Range:
Days-Prior Range  Av

/tmp/ipython-input-2059195548.py:85: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df.groupby(['hotel_code', 'days_prior_range'])


In [ ]:
data_path = "hotel_bookings.csv"

asof_start = pd.Timestamp("2010-01-30")
asof_end = pd.Timestamp("2010-03-31")
test_period_start = pd.Timestamp("2010-01-01")
test_period_end = pd.Timestamp("2010-04-30")
sequence_length = 60
num_forecast_steps = 30
batch_size = 64
num_epochs = 60
learning_rate = 1e-2
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

grid = [
    {'hidden_size': 30, 'num_layers': 1, 'dropout': 0.0},
    {'hidden_size': 60, 'num_layers': 1, 'dropout': 0.0},
    {'hidden_size': 60, 'num_layers': 2, 'dropout': 0.2},
    {'hidden_size': 90, 'num_layers': 2, 'dropout': 0.3},
    {'hidden_size': 120, 'num_layers': 3, 'dropout': 0.3},
    {'hidden_size': 64, 'num_layers': 1, 'dropout': 0.0},
    {'hidden_size': 64, 'num_layers': 2, 'dropout': 0.2},
]

# Evaluation Metrics
def mae(y_true, y_pred):
    return np.nanmean(np.abs(y_true - y_pred))

def mape(y_true, y_pred):
    mask = (~np.isnan(y_true)) & (~np.isnan(y_pred)) & (y_true != 0)
    if mask.sum() == 0:
        return np.nan
    return np.nanmean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100.0

def mase(y_true, y_pred, y_naive):
    mask = (~np.isnan(y_true)) & (~np.isnan(y_pred))
    if mask.sum() == 0:
        return np.nan
    y_true_valid = y_true[mask]
    y_pred_valid = y_pred[mask]
    y_naive_valid = y_naive[mask]
    naive_mask = ~np.isnan(y_naive_valid)
    if naive_mask.sum() == 0:
        return np.nan
    mae_naive = np.nanmean(np.abs(y_true_valid[naive_mask] - y_naive_valid[naive_mask]))
    if mae_naive == 0 or np.isnan(mae_naive):
        return np.nan
    mae_model = np.nanmean(np.abs(y_true_valid - y_pred_valid))
    return mae_model / mae_naive

# Define LSTM Model
class LSTMModel(nn.Module):
    def __init__(self, input_size=1, hidden_size=64, num_layers=1, dropout=0.0):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(
            input_size,
            hidden_size,
            num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.linear = nn.Linear(hidden_size, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        return self.linear(out[:, -1, :])

# Data Helpers
def create_sequences(scaled_data, seq_len):
    x, y = [], []
    for i in range(len(scaled_data) - seq_len):
        x.append(scaled_data[i:i + seq_len])
        y.append(scaled_data[i + seq_len])
    x = np.array(x).reshape((-1, seq_len, 1))
    y = np.array(y).reshape((-1, 1))
    return x, y

def build_daily_series(df_hotel):
    idx = pd.date_range(df_hotel['stay_date'].min(), df_hotel['stay_date'].max(), freq='D')
    series = pd.Series(0.0, index=idx)
    final_demand = df_hotel.groupby('stay_date')['final_demand'].last()
    series.loc[final_demand.index] = final_demand.values
    return series.sort_index()

# Processing
df = pd.read_csv(data_path)
df.columns = [c.strip().lower() for c in df.columns]
required_cols = {'hotel_code', 'stay_date', 'final_demand'}
if not required_cols.issubset(set(df.columns)):
    raise ValueError(f"Missing columns: {required_cols - set(df.columns)}")

df['stay_date'] = pd.to_datetime(df['stay_date'])
hotels = sorted(df['hotel_code'].unique())
print(f"Hotels: {hotels}\n")

all_forecasts = []
train_results = []
test_results = []

# Loop through hotels
for hotel in hotels:
    print(f"HOTEL: {hotel}")
    df_hotel = df[df['hotel_code'] == hotel].copy()
    daily = build_daily_series(df_hotel)
    daily.index.name = 'stay_date'

    # Create Naive forecast map
    naive_map = {}
    for d in daily.index:
        prev = d - pd.Timedelta(days=364)
        naive_map[d] = daily.loc[prev] if prev in daily.index else np.nan

    # Train split
    train_mask = daily.index < pd.Timestamp("2010-01-01")
    train_series = daily[train_mask].values.astype(float)
    train_dates = daily.index[train_mask]

    if len(train_series) < sequence_length + 1:
        continue

    # Scale
    scaler = StandardScaler().fit(train_series.reshape(-1, 1))
    scaled_train = scaler.transform(train_series.reshape(-1, 1)).flatten()

    # Create Sequences
    x_all, y_all = create_sequences(scaled_train, sequence_length)
    n_samples = x_all.shape[0]
    val_size = max(1, int(0.1 * n_samples))
    train_size = n_samples - val_size
    x_train, y_train = x_all[:train_size], y_all[:train_size]
    x_val, y_val = x_all[train_size:], y_all[train_size:]

    def make_loader(x, y, shuffle=True):
        dataset = TensorDataset(torch.tensor(x, dtype=torch.float32), torch.tensor(y, dtype=torch.float32))
        return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle)

    train_loader = make_loader(x_train, y_train, True)
    val_loader = make_loader(x_val, y_val, False)

    # Grid search
    print(f"\n Finding best architecture:")
    best_config = None
    best_val_mae = np.inf
    best_state = None

    for config in grid:
        model = LSTMModel(input_size=1, **config).to(device)
        optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
        loss_fn = nn.MSELoss()

        # Quick training with 15-epoch
        for _ in range(15):
            model.train()
            for bx, by in train_loader:
                bx, by = bx.to(device), by.to(device)
                pred = model(bx)
                loss = loss_fn(pred, by)
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

        # Validate
        model.eval()
        with torch.no_grad():
            val_preds = np.concatenate([model(bx.to(device)).cpu().numpy().ravel() for bx, _ in val_loader])
            val_trues = np.concatenate([by.numpy().ravel() for _, by in val_loader])
            val_mae_score = mae(val_trues, val_preds)

        print(f" Hidden={config['hidden_size']:3d}, Layers={config['num_layers']}, "
              f"Dropout={config['dropout']:.1f} → VAL_MAE={val_mae_score:.4f}")

        if val_mae_score < best_val_mae:
            best_val_mae = val_mae_score
            best_config = config
            best_state = model.state_dict()

    print(f" Best architecture: Hidden={best_config['hidden_size']}, "
          f"Layers={best_config['num_layers']}, VAL_MAE={best_val_mae:.4f}\n")

    # Final training with best configuration
    model = LSTMModel(input_size=1, **best_config).to(device)
    if best_state is not None:
        model.load_state_dict(best_state)

    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    loss_fn = nn.MSELoss()
    full_loader = make_loader(x_all, y_all, True)

    print(f" Training final model ({num_epochs} epochs)...")
    for epoch in range(num_epochs):
        model.train()
        losses = []
        for bx, by in full_loader:
            bx, by = bx.to(device), by.to(device)
            pred = model(bx)
            loss = loss_fn(pred, by)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            losses.append(loss.item())
        if (epoch + 1) % 20 == 0:
            print(f" Epoch {epoch+1}/{num_epochs} - MSE Loss: {np.mean(losses):.6f}")

    #  In-sample
    model.eval()
    with torch.no_grad():
        train_preds_scaled = model(torch.tensor(x_all, dtype=torch.float32).to(device)).cpu().numpy().ravel()
    train_preds = scaler.inverse_transform(train_preds_scaled.reshape(-1, 1)).ravel()
    train_actual = train_series[sequence_length:]
    train_naive = np.array([naive_map.get(d, np.nan) for d in train_dates[sequence_length:]])

    train_mae_val = mae(train_actual, train_preds)
    train_mape_val = mape(train_actual, train_preds)
    train_mase_val = mase(train_actual, train_preds, train_naive)
    train_results.append({'hotel': hotel, 'mae': train_mae_val, 'mape': train_mape_val, 'mase': train_mase_val})

    # Forecasting
    forecast_rows = []
    asof = asof_start
    while asof <= asof_end:
        if asof not in daily.index:
            asof += timedelta(days=1)
            continue
        history = daily.loc[:asof].values.astype(float)
        if len(history) < sequence_length:
            asof += timedelta(days=1)
            continue

        hist_scaled = scaler.transform(history.reshape(-1, 1)).flatten()
        seq = hist_scaled[-sequence_length:].copy()

        preds_scaled = []
        model.eval()
        with torch.no_grad():
            for _ in range(num_forecast_steps):
                x = torch.tensor(seq).view(1, -1, 1).float().to(device)
                pred = model(x).cpu().numpy()[0, 0]
                preds_scaled.append(pred)
                seq = np.roll(seq, -1)
                seq[-1] = pred

        preds = scaler.inverse_transform(np.array(preds_scaled).reshape(-1, 1)).ravel()
        for h, f in enumerate(preds, 1):
            stay = asof + timedelta(days=h)
            actual = daily.loc[stay] if stay in daily.index else np.nan
            naive = naive_map.get(stay, np.nan)
            forecast_rows.append({
                'hotel': hotel,
                'as_of_date': asof,
                'stay_date': stay,
                'days_prior': h,
                'forecast': float(f),
                'actual': float(actual) if not pd.isna(actual) else np.nan,
                'naive': float(naive) if not pd.isna(naive) else np.nan
            })
        asof += timedelta(days=1)

    df_forecasts = pd.DataFrame(forecast_rows)
    all_forecasts.append(df_forecasts)

    print(f"\nSample Forecasts for Hotel {hotel}:")
    head = df_forecasts[['as_of_date', 'stay_date', 'days_prior', 'forecast']].head(15)
    for _, r in head.iterrows():
        print(f"{r['as_of_date'].date()}  {r['stay_date'].date()}  {r['days_prior']:2}  {r['forecast']:.6f}")
    print("...")
    tail = df_forecasts[['as_of_date', 'stay_date', 'days_prior', 'forecast']].tail(15)
    for _, r in tail.iterrows():
        print(f"{r['as_of_date'].date()}  {r['stay_date'].date()}  {r['days_prior']:2}  {r['forecast']:.6f}")

# Out-of-sample evaluation
df_all = pd.concat(all_forecasts, ignore_index=True)
eval_mask = (df_all['stay_date'] >= test_period_start) & (df_all['stay_date'] <= test_period_end)
df_eval = df_all[eval_mask].copy()
df_eval = df_eval.dropna(subset=['actual']).copy()
df_eval['abs_error'] = np.abs(df_eval['actual'] - df_eval['forecast'])
df_eval['ape'] = np.where(df_eval['actual'] != 0, df_eval['abs_error'] / df_eval['actual'], np.nan)
df_eval['month_str'] = df_eval['stay_date'].dt.strftime('%m-%Y')

# Overall out-of-sample metrics
overall_mae = np.nanmean(df_eval['abs_error'])
overall_mape = np.nanmean(df_eval['ape']) * 100

mase_values = []
for hotel in hotels:
    hotel_eval = df_eval[df_eval['hotel'] == hotel]
    if hotel_eval.empty:
        continue
    train_h = df[df['hotel_code'] == hotel]
    train_mask = train_h['stay_date'] < pd.Timestamp("2010-01-01")
    train_daily = build_daily_series(train_h[train_mask])
    naive_train = [naive_map.get(d, np.nan) for d in train_daily.index]
    denom = np.nanmean(np.abs(train_daily.values - np.array(naive_train)))
    if np.isnan(denom) or denom == 0:
        continue
    hotel_mae = np.nanmean(hotel_eval['abs_error'])
    hotel_mase = hotel_mae / denom
    mase_values.append(hotel_mase)

overall_mase = np.mean(mase_values) if mase_values else np.nan

print(f"\nOUT-OF-SAMPLE PERFORMANCE (TEST PERIOD: {test_period_start.date()} to {test_period_end.date()})")
print(f"{'Metric':<8} {'Value':>10}")
print("-" * 25)
print(f"{'MAE':<8} {overall_mae:>10.2f}")
print(f"{'MAPE':<8} {overall_mape:>9.2f}%")
print(f"{'MASE':<8} {overall_mase:>10.4f}")

# Forecast by Months and Days-Prior Range
months = ['01-2010', '02-2010', '03-2010', '04-2010']
ranges = ['1-7', '8-14', '15-21', '22-30']
bins = [0, 7, 14, 21, 30]

for hotel in hotels:
    print(f"\nHOTEL {hotel}")
    hotel_eval = df_eval[df_eval['hotel'] == hotel].copy()
    if hotel_eval.empty:
        continue

    train_h = df[df['hotel_code'] == hotel]
    train_mask = train_h['stay_date'] < pd.Timestamp("2010-01-01")
    train_daily = build_daily_series(train_h[train_mask])
    naive_train = [naive_map.get(d, np.nan) for d in train_daily.index]
    denom = np.nanmean(np.abs(train_daily.values - np.array(naive_train)))
    if np.isnan(denom) or denom == 0:
        denom = np.nan

    print("Average Errors by Month:")
    print(f"{'Month':<10} {'MAE':>8} {'MAPE':>10} {'MASE':>10}")
    print("-" * 40)
    for m in months:
        g = hotel_eval[hotel_eval['month_str'] == m]
        if len(g) == 0: continue
        mae_v = np.nanmean(g['abs_error'])
        mape_v = np.nanmean(g['ape']) * 100
        mase_v = mae_v / denom if not np.isnan(denom) and denom > 0 else np.nan
        print(f"{m:<10} {mae_v:>8.2f} {mape_v:>10.2f} {mase_v:>10.4f}")

    hotel_eval['range'] = pd.cut(hotel_eval['days_prior'], bins=bins, labels=ranges, include_lowest=True)
    print("\nAverage Errors by Days-Prior Range:")
    print(f"{'Range':<10} {'MAE':>8} {'MAPE':>10} {'MASE':>10}")
    print("-" * 40)
    for r in ranges:
        g = hotel_eval[hotel_eval['range'] == r]
        if len(g) == 0: continue
        mae_v = np.nanmean(g['abs_error'])
        mape_v = np.nanmean(g['ape']) * 100
        mase_v = mae_v / denom if not np.isnan(denom) and denom > 0 else np.nan
        print(f"{r:<10} {mae_v:>8.2f} {mape_v:>10.2f} {mase_v:>10.4f}")

# In-sample Performance
print("\nIN-SAMPLE PERFORMANCE (TRAIN)")
print(f"{'Hotel':<8} {'MAE':>8} {'MAPE':>10} {'MASE':>10}")
print("-" * 40)
for r in train_results:
    print(f"{r['hotel']:<8} {r['mae']:>8.2f} {r['mape']:>10.2f} {r['mase']:>10.4f}")

avg_mae = np.mean([r['mae'] for r in train_results])
avg_mape = np.mean([r['mape'] for r in train_results])
avg_mase = np.mean([r['mase'] for r in train_results])
print(f"{'AVERAGE':<8} {avg_mae:>8.2f} {avg_mape:>10.2f} {avg_mase:>10.4f}")

Hotels: ['G', 'M', 'W']

HOTEL: G

 Finding best architecture:
 Hidden= 30, Layers=1, Dropout=0.0 → VAL_MAE=0.7618
 Hidden= 60, Layers=1, Dropout=0.0 → VAL_MAE=0.6990
 Hidden= 60, Layers=2, Dropout=0.2 → VAL_MAE=0.7180
 Hidden= 90, Layers=2, Dropout=0.3 → VAL_MAE=0.6660
 Hidden=120, Layers=3, Dropout=0.3 → VAL_MAE=0.6819
 Hidden= 64, Layers=1, Dropout=0.0 → VAL_MAE=0.7553
 Hidden= 64, Layers=2, Dropout=0.2 → VAL_MAE=0.7481
 Best architecture: Hidden=90, Layers=2, VAL_MAE=0.6660

 Training final model (60 epochs)...
 Epoch 20/60 - MSE Loss: 0.352161
 Epoch 40/60 - MSE Loss: 0.139518
 Epoch 60/60 - MSE Loss: 0.036212

Sample Forecasts for Hotel G:
2010-01-30  2010-01-31   1  64.071396
2010-01-30  2010-02-01   2  96.177612
2010-01-30  2010-02-02   3  87.993416
2010-01-30  2010-02-03   4  85.750366
2010-01-30  2010-02-04   5  95.585693
2010-01-30  2010-02-05   6  110.133102
2010-01-30  2010-02-06   7  129.700256
2010-01-30  2010-02-07   8  49.848934
2010-01-30  2010-02-08   9  87.808487
20